# Lab 3.3.4 Solutions: Medusa Speculative Decoding

Complete solutions to all exercises from the Medusa speculative decoding lab.

## Setup

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path("../scripts").resolve()))

import numpy as np
import time
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import requests

---

## Exercise 1: Implement Medusa Head Simulator

**Task**: Create a simulation of Medusa heads to understand the tree attention mechanism.

In [ ]:
@dataclass
class MedusaHeadConfig:
    """Configuration for Medusa heads."""
    num_heads: int = 5           # Number of additional prediction heads
    top_k_per_head: int = 10     # Top-k candidates per head
    tree_depth: int = 5          # Maximum speculation depth


class MedusaSimulator:
    """
    Simulate Medusa speculative decoding to understand its mechanics.
    
    Medusa adds multiple prediction heads that each predict future tokens.
    Head 0: predicts token t+1 (same as base model)
    Head 1: predicts token t+2
    Head 2: predicts token t+3
    ...
    
    These form a tree of possible continuations.
    """
    
    def __init__(self, config: MedusaHeadConfig = None):
        self.config = config or MedusaHeadConfig()
        self.vocabulary = self._create_mock_vocabulary()
        
    def _create_mock_vocabulary(self, size: int = 1000) -> List[str]:
        """Create a mock vocabulary for simulation."""
        common_words = [
            "the", "a", "is", "are", "was", "were", "be", "been",
            "to", "of", "and", "in", "that", "it", "for", "on",
            "with", "as", "at", "by", "from", "or", "an", "but",
            "not", "this", "have", "has", "had", "do", "does", "did",
            "will", "would", "could", "should", "may", "might", "can",
            "AI", "model", "data", "learning", "neural", "network",
            "training", "inference", "token", "embedding", "layer",
        ]
        return common_words + [f"word_{i}" for i in range(size - len(common_words))]
    
    def simulate_head_predictions(self, 
                                   context: str,
                                   num_heads: int = None) -> Dict[int, List[Tuple[str, float]]]:
        """
        Simulate predictions from each Medusa head.
        
        Returns: {head_idx: [(token, probability), ...]}
        """
        num_heads = num_heads or self.config.num_heads
        predictions = {}
        
        for head_idx in range(num_heads):
            # Simulate probability distribution
            # Later heads are less confident (entropy increases with distance)
            confidence_decay = 0.8 ** head_idx
            
            # Generate top-k predictions for this head
            probs = np.random.dirichlet(np.ones(self.config.top_k_per_head) * confidence_decay)
            probs = sorted(probs, reverse=True)
            
            tokens = np.random.choice(
                self.vocabulary[:100],  # Use common words
                size=self.config.top_k_per_head,
                replace=False
            )
            
            predictions[head_idx] = list(zip(tokens, probs))
        
        return predictions
    
    def build_speculation_tree(self, 
                                head_predictions: Dict[int, List[Tuple[str, float]]],
                                max_candidates: int = 64) -> List[List[str]]:
        """
        Build tree of candidate sequences from head predictions.
        
        This is the key Medusa innovation: tree attention allows
        verifying multiple paths in parallel.
        """
        # Start with top-1 from each head (the "main path")
        candidates = []
        
        # Build tree using top predictions
        def build_paths(depth: int, current_path: List[str], 
                        cumulative_prob: float):
            if depth >= len(head_predictions):
                if current_path:
                    candidates.append((current_path.copy(), cumulative_prob))
                return
            
            # Take top candidates from this head
            top_k = min(3, len(head_predictions[depth]))  # Limit branching
            for token, prob in head_predictions[depth][:top_k]:
                current_path.append(token)
                build_paths(depth + 1, current_path, cumulative_prob * prob)
                current_path.pop()
            
            # Also include partial path (stop early)
            if current_path:
                candidates.append((current_path.copy(), cumulative_prob))
        
        build_paths(0, [], 1.0)
        
        # Sort by probability and take top candidates
        candidates.sort(key=lambda x: x[1], reverse=True)
        return [c[0] for c in candidates[:max_candidates]]
    
    def simulate_verification(self, 
                               candidates: List[List[str]],
                               acceptance_rate: float = 0.7) -> Tuple[List[str], int]:
        """
        Simulate the verification step.
        
        In real Medusa, the base model verifies all candidates in parallel
        using tree attention, accepting the longest matching prefix.
        """
        if not candidates:
            return [], 0
        
        # Simulate: each token has acceptance_rate probability
        best_accepted = []
        
        for candidate in candidates:
            accepted = []
            for token in candidate:
                if np.random.random() < acceptance_rate:
                    accepted.append(token)
                else:
                    break  # Stop at first rejection
            
            if len(accepted) > len(best_accepted):
                best_accepted = accepted
        
        return best_accepted, len(best_accepted)


# Demonstrate Medusa mechanics
print("🐍 Medusa Speculative Decoding Simulation")
print("=" * 60)

simulator = MedusaSimulator(MedusaHeadConfig(
    num_heads=5,
    top_k_per_head=5,
    tree_depth=5
))

# Simulate one step
context = "The future of AI is"
print(f"\nContext: '{context}'")

# Get head predictions
predictions = simulator.simulate_head_predictions(context)

print("\n📊 Head Predictions (top-3 per head):")
for head_idx, preds in predictions.items():
    top_3 = [(t, f"{p:.2f}") for t, p in preds[:3]]
    print(f"  Head {head_idx} (t+{head_idx+1}): {top_3}")

# Build speculation tree
candidates = simulator.build_speculation_tree(predictions, max_candidates=20)
print(f"\n🌲 Speculation Tree: {len(candidates)} candidate sequences")
print("  Top-5 candidates:")
for i, cand in enumerate(candidates[:5]):
    print(f"    {i+1}. {' → '.join(cand)}")

# Simulate verification
accepted, num_accepted = simulator.simulate_verification(candidates, acceptance_rate=0.7)
print(f"\n✅ Verification Result:")
print(f"  Accepted: {' → '.join(accepted) if accepted else '(none)'}")
print(f"  Tokens accepted: {num_accepted}")
print(f"  Speedup: ~{max(1, num_accepted)}x (generated {num_accepted} tokens in 1 forward pass)")

---

## Exercise 2: Calculate Optimal Head Configuration

**Task**: Analyze how different head configurations affect speedup and accuracy.

In [ ]:
@dataclass
class HeadAnalysisResult:
    """Results from analyzing a Medusa head configuration."""
    num_heads: int
    avg_accepted: float
    theoretical_speedup: float
    overhead_factor: float
    effective_speedup: float
    memory_overhead_mb: float


def analyze_head_configuration(num_heads: int,
                                base_acceptance_rate: float = 0.8,
                                decay_rate: float = 0.9,
                                head_params_mb: float = 50,
                                overhead_per_head: float = 0.05) -> HeadAnalysisResult:
    """
    Analyze a Medusa head configuration.
    
    Parameters:
        num_heads: Number of speculative heads
        base_acceptance_rate: Probability first head is correct
        decay_rate: How much acceptance drops per head
        head_params_mb: Memory per head (parameters)
        overhead_per_head: Computational overhead per head
    """
    # Calculate expected tokens accepted
    # Each head has acceptance = base * decay^head_idx
    expected_accepted = 0
    cumulative_prob = 1.0
    
    for i in range(num_heads):
        head_acceptance = base_acceptance_rate * (decay_rate ** i)
        cumulative_prob *= head_acceptance
        expected_accepted += cumulative_prob
    
    # Add 1 for the base model token (always generated)
    avg_accepted = 1 + expected_accepted
    
    # Theoretical speedup = tokens per forward pass
    theoretical_speedup = avg_accepted
    
    # Overhead from running additional heads
    overhead_factor = 1 + (num_heads * overhead_per_head)
    
    # Effective speedup accounts for overhead
    effective_speedup = theoretical_speedup / overhead_factor
    
    # Memory overhead
    memory_overhead = num_heads * head_params_mb
    
    return HeadAnalysisResult(
        num_heads=num_heads,
        avg_accepted=avg_accepted,
        theoretical_speedup=theoretical_speedup,
        overhead_factor=overhead_factor,
        effective_speedup=effective_speedup,
        memory_overhead_mb=memory_overhead
    )


def find_optimal_configuration(max_heads: int = 10,
                                max_memory_mb: float = 500) -> Tuple[int, HeadAnalysisResult]:
    """
    Find the optimal number of Medusa heads.
    
    Trade-offs:
    - More heads = more speculation = higher potential speedup
    - More heads = more overhead = diminishing returns
    - More heads = more memory usage
    """
    results = []
    
    for num_heads in range(1, max_heads + 1):
        result = analyze_head_configuration(num_heads)
        
        # Skip if exceeds memory budget
        if result.memory_overhead_mb > max_memory_mb:
            break
        
        results.append(result)
    
    # Find best effective speedup
    best = max(results, key=lambda x: x.effective_speedup)
    return best.num_heads, best


# Analyze different configurations
print("📊 Medusa Head Configuration Analysis")
print("=" * 70)
print(f"{'Heads':>6} {'Avg Accepted':>14} {'Theoretical':>12} {'Overhead':>10} {'Effective':>10} {'Memory':>10}")
print("-" * 70)

for num_heads in range(1, 11):
    result = analyze_head_configuration(num_heads)
    marker = "←" if result.num_heads == 5 else ""
    print(f"{result.num_heads:>6} {result.avg_accepted:>14.2f} "
          f"{result.theoretical_speedup:>11.2f}x {result.overhead_factor:>9.2f}x "
          f"{result.effective_speedup:>9.2f}x {result.memory_overhead_mb:>8.0f}MB {marker}")

optimal_heads, optimal_result = find_optimal_configuration()
print(f"\n✅ Optimal Configuration: {optimal_heads} heads")
print(f"   Effective speedup: {optimal_result.effective_speedup:.2f}x")
print(f"   Memory overhead: {optimal_result.memory_overhead_mb:.0f}MB")

print("\n💡 Key Insights:")
print("   • 4-6 heads typically optimal for most models")
print("   • Beyond 6 heads, overhead outweighs benefit")
print("   • Acceptance rate drops ~10-20% per head")
print("   • Consider memory budget for production")

---

## Exercise 3: Benchmark Different Task Types

**Task**: Measure how Medusa performs on different types of tasks (creative vs factual).

In [ ]:
@dataclass 
class TaskTypeResult:
    """Benchmark results for a specific task type."""
    task_type: str
    avg_acceptance_rate: float
    avg_tokens_per_step: float
    effective_speedup: float
    sample_prompt: str


# Task type characteristics
TASK_CHARACTERISTICS = {
    "code_completion": {
        "description": "Complete code snippets",
        "expected_acceptance": 0.85,  # High - code is predictable
        "temperature": 0.1,
        "prompts": [
            "def fibonacci(n):\n    if n <= 1:\n        return n\n    return",
            "class DataLoader:\n    def __init__(self, data):\n        self.data = data\n    def __len__(self):",
            "import numpy as np\n\ndef matrix_multiply(a, b):\n    return",
        ]
    },
    "factual_qa": {
        "description": "Answer factual questions",
        "expected_acceptance": 0.75,  # Medium-high - common knowledge
        "temperature": 0.3,
        "prompts": [
            "What is the capital of France? The capital is",
            "Python was created by Guido van Rossum in",
            "The speed of light is approximately",
        ]
    },
    "summarization": {
        "description": "Summarize text",
        "expected_acceptance": 0.70,  # Medium - some variation
        "temperature": 0.5,
        "prompts": [
            "Summarize: Machine learning is a subset of AI. TL;DR:",
            "The key points are:",
            "In summary,",
        ]
    },
    "creative_writing": {
        "description": "Creative/open-ended writing",
        "expected_acceptance": 0.55,  # Lower - many valid continuations
        "temperature": 0.9,
        "prompts": [
            "Once upon a time in a galaxy far away,",
            "Write a poem about the ocean:",
            "Imagine a world where robots",
        ]
    },
    "translation": {
        "description": "Translate between languages",
        "expected_acceptance": 0.80,  # High - deterministic mapping
        "temperature": 0.2,
        "prompts": [
            "Translate to French: Hello, how are you?\nFrench:",
            "English: The cat is on the mat.\nSpanish:",
            "日本語: おはようございます\nEnglish:",
        ]
    }
}


def simulate_task_benchmark(task_type: str, 
                            num_samples: int = 100,
                            num_heads: int = 5) -> TaskTypeResult:
    """
    Simulate Medusa performance on a specific task type.
    """
    config = TASK_CHARACTERISTICS[task_type]
    base_acceptance = config["expected_acceptance"]
    
    # Simulate multiple generation steps
    total_accepted = 0
    total_steps = 0
    
    for _ in range(num_samples):
        # Simulate one generation step
        accepted = 0
        cumulative_prob = 1.0
        
        for head_idx in range(num_heads):
            # Acceptance decreases with each head
            head_acceptance = base_acceptance * (0.9 ** head_idx)
            # Add some randomness
            head_acceptance *= np.random.uniform(0.9, 1.1)
            
            if np.random.random() < head_acceptance * cumulative_prob:
                accepted += 1
                cumulative_prob *= head_acceptance
            else:
                break
        
        total_accepted += accepted + 1  # +1 for base model token
        total_steps += 1
    
    avg_tokens = total_accepted / total_steps
    
    return TaskTypeResult(
        task_type=task_type,
        avg_acceptance_rate=base_acceptance,
        avg_tokens_per_step=avg_tokens,
        effective_speedup=avg_tokens / 1.25,  # Account for overhead
        sample_prompt=config["prompts"][0][:50] + "..."
    )


# Run benchmarks for all task types
print("📊 Medusa Performance by Task Type")
print("=" * 70)
print(f"{'Task Type':<20} {'Accept Rate':>12} {'Tokens/Step':>12} {'Speedup':>10}")
print("-" * 70)

results = []
for task_type in TASK_CHARACTERISTICS:
    result = simulate_task_benchmark(task_type)
    results.append(result)
    
    bar = "█" * int(result.effective_speedup * 3)
    print(f"{task_type:<20} {result.avg_acceptance_rate:>11.0%} "
          f"{result.avg_tokens_per_step:>12.1f} {result.effective_speedup:>9.2f}x {bar}")

# Analysis
best = max(results, key=lambda x: x.effective_speedup)
worst = min(results, key=lambda x: x.effective_speedup)

print(f"\n✅ Best performance: {best.task_type} ({best.effective_speedup:.2f}x)")
print(f"⚠️  Worst performance: {worst.task_type} ({worst.effective_speedup:.2f}x)")

print("\n💡 Key Insights:")
print("   • Code completion benefits most (predictable patterns)")
print("   • Creative writing benefits least (high entropy)")
print("   • Consider temperature: lower temp = better speculation")
print("   • Task-aware routing can optimize overall throughput")

---

## Exercise 4: Implement Adaptive Speculation Depth

**Task**: Create a system that adjusts speculation depth based on acceptance rates.

In [ ]:
class AdaptiveSpeculationController:
    """
    Dynamically adjust Medusa speculation depth based on observed acceptance.
    
    Key idea: If acceptance rate is high, use more heads.
    If acceptance rate is low, use fewer heads to reduce overhead.
    """
    
    def __init__(self,
                 min_heads: int = 2,
                 max_heads: int = 8,
                 target_efficiency: float = 0.6,
                 window_size: int = 50):
        self.min_heads = min_heads
        self.max_heads = max_heads
        self.target_efficiency = target_efficiency
        self.window_size = window_size
        
        self.current_heads = (min_heads + max_heads) // 2
        self.acceptance_history: List[float] = []
        self.heads_history: List[int] = []
    
    def record_step(self, tokens_proposed: int, tokens_accepted: int):
        """
        Record the result of a speculation step.
        
        tokens_proposed: How many tokens we tried to generate
        tokens_accepted: How many were actually correct
        """
        acceptance_rate = tokens_accepted / tokens_proposed if tokens_proposed > 0 else 0
        self.acceptance_history.append(acceptance_rate)
        self.heads_history.append(self.current_heads)
        
        # Keep window limited
        if len(self.acceptance_history) > self.window_size:
            self.acceptance_history.pop(0)
            self.heads_history.pop(0)
        
        # Adjust speculation depth
        self._adjust_depth()
    
    def _adjust_depth(self):
        """Adjust the number of speculation heads based on recent performance."""
        if len(self.acceptance_history) < 10:
            return  # Need more data
        
        # Calculate efficiency: accepted / (proposed * overhead)
        recent_acceptance = np.mean(self.acceptance_history[-10:])
        overhead = 1 + (self.current_heads * 0.05)
        efficiency = recent_acceptance / overhead
        
        # Adjust based on efficiency vs target
        if efficiency > self.target_efficiency * 1.1:  # 10% above target
            # Acceptance is good, try more heads
            self.current_heads = min(self.max_heads, self.current_heads + 1)
        elif efficiency < self.target_efficiency * 0.9:  # 10% below target
            # Acceptance is poor, use fewer heads
            self.current_heads = max(self.min_heads, self.current_heads - 1)
    
    def get_speculation_config(self) -> dict:
        """Get current speculation configuration."""
        recent_acceptance = np.mean(self.acceptance_history[-10:]) if len(self.acceptance_history) >= 10 else 0.5
        
        return {
            "num_heads": self.current_heads,
            "recent_acceptance": recent_acceptance,
            "samples_collected": len(self.acceptance_history)
        }
    
    def get_stats(self) -> dict:
        """Get detailed statistics."""
        if not self.acceptance_history:
            return {"status": "no data"}
        
        return {
            "total_steps": len(self.acceptance_history),
            "current_heads": self.current_heads,
            "avg_acceptance": np.mean(self.acceptance_history),
            "recent_acceptance": np.mean(self.acceptance_history[-10:]),
            "heads_changes": len(set(self.heads_history)),
            "avg_heads_used": np.mean(self.heads_history)
        }


# Simulate adaptive speculation
print("🎯 Adaptive Speculation Controller Demo")
print("=" * 60)

controller = AdaptiveSpeculationController(
    min_heads=2,
    max_heads=8,
    target_efficiency=0.6
)

# Simulate different phases
print("\nSimulating generation with varying difficulty...\n")

phases = [
    ("Code completion (easy)", 0.85, 30),
    ("Technical writing (medium)", 0.70, 30),
    ("Creative writing (hard)", 0.50, 30),
    ("Back to code (easy)", 0.85, 30),
]

print(f"{'Phase':<30} {'Acceptance':>12} {'Heads Used':>12}")
print("-" * 55)

for phase_name, base_acceptance, steps in phases:
    phase_heads = []
    
    for _ in range(steps):
        # Simulate with current configuration
        config = controller.get_speculation_config()
        proposed = config["num_heads"]
        
        # Simulate acceptance (with noise)
        acceptance = base_acceptance * np.random.uniform(0.8, 1.2)
        accepted = int(proposed * min(1.0, acceptance))
        
        controller.record_step(proposed, accepted)
        phase_heads.append(config["num_heads"])
    
    avg_heads = np.mean(phase_heads)
    print(f"{phase_name:<30} {base_acceptance:>11.0%} {avg_heads:>12.1f}")

# Final stats
stats = controller.get_stats()
print(f"\n📊 Final Statistics:")
print(f"   Total steps: {stats['total_steps']}")
print(f"   Average heads used: {stats['avg_heads_used']:.1f}")
print(f"   Overall acceptance: {stats['avg_acceptance']:.1%}")

print("\n💡 Observation: Controller automatically adjusts heads")
print("   based on content difficulty!")

---

## Key Takeaways

1. **Medusa Architecture**: Multiple prediction heads speculate future tokens in parallel

2. **Tree Attention**: Verify multiple candidate paths in a single forward pass

3. **Optimal Heads**: 4-6 heads typically best; beyond that, overhead outweighs benefit

4. **Task Dependence**: 
   - Code completion: High acceptance, big speedup
   - Creative writing: Low acceptance, modest speedup

5. **Adaptive Depth**: Dynamically adjust based on observed acceptance rates

6. **Production Tip**: Match speculation depth to your workload characteristics